In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder as lb
from sklearn.svm import SVC
%matplotlib inline

In [7]:
input_data_train = pd.read_csv("E:/ML assignment/incomedata/income-data/income.train.txt",header = None)
input_data_test = pd.read_csv("E:/ML assignment/incomedata/income-data/income.test.txt",header = None)
input_data_dev = pd.read_csv("E:/ML assignment/incomedata/income-data/income.dev.txt",header = None)

In [8]:
input_data_train.head()
input_data_dev.head()
input_data_test.head()

,0,1,2,3,4,5,6,7,8,9
0,47,Private,11th,Married-civ-spouse,Craft-repair,White,Male,40,Dominican-Republic,<=50K
1,20,Private,Some-college,Never-married,Other-service,White,Female,25,United-States,<=50K
2,45,Local-gov,HS-grad,Married-civ-spouse,Transport-moving,White,Female,40,United-States,>50K
3,45,Local-gov,Masters,Married-civ-spouse,Prof-specialty,White,Male,37,United-States,>50K
4,27,State-gov,Bachelors,Never-married,Prof-specialty,Black,Male,40,United-States,<=50K


In [9]:
combined_data = pd.concat([input_data_train,input_data_test,input_data_dev],keys = ["train","test","dev"] )
combined_data.head()

0                  1           2                    3  \
train 0  50   Self-emp-not-inc   Bachelors   Married-civ-spouse   
      1  38            Private     HS-grad             Divorced   
      2  53            Private        11th   Married-civ-spouse   
      3  28            Private   Bachelors   Married-civ-spouse   
      4  37            Private     Masters   Married-civ-spouse   

                          4       5        6   7               8       9  
train 0     Exec-managerial   White     Male  13   United-States   <=50K  
      1   Handlers-cleaners   White     Male  40   United-States   <=50K  
      2   Handlers-cleaners   Black     Male  40   United-States   <=50K  
      3      Prof-specialty   Black   Female  40            Cuba   <=50K  
      4     Exec-managerial   White   Female  40   United-States   <=50K

In [10]:
# One hot encoding except to 9 column
combined_data = pd.get_dummies(combined_data,columns=[1,2,3,4,5,6,8])


In [11]:
X_Train = combined_data.xs("train")
X_Test = combined_data.xs("test")
X_Dev = combined_data.xs("dev")

# print(X_Train.head())
# print(X_Test.head())
# print(X_Dev.head())

# Extract Y train values from X data 

Y_Train = X_Train[9]
Y_Test = X_Test[9]
Y_Dev = X_Dev[9]

X_Train = X_Train.drop([9], axis = 1)
X_Test = X_Test.drop([9],axis = 1)
X_Dev = X_Dev.drop([9],axis = 1)

# Label Encoding on Y
labelenco = lb() 
Y_Train = labelenco.fit_transform(Y_Train)
Y_Test = labelenco.fit_transform(Y_Test)
Y_Dev = labelenco.fit_transform(Y_Dev)

# Bring Zeros to -1 

Y_Train[ Y_Train == 0] = -1
Y_Test[ Y_Test == 0] = -1
Y_Dev[ Y_Dev == 0] = -1

X_Train = X_Train.iloc[:,:].values
X_Test = X_Test.iloc[:,:].values
X_Dev = X_Dev.iloc[:,:].values

print(Y_Train)
print(Y_Test)
print(Y_Dev)

[-1 -1 -1 ... -1 -1  1]
[-1 -1  1 ... -1  1  1]
[ 1 -1 -1 ... -1 -1 -1]


In [12]:
# Drop the last Y column form the code


In [13]:
print(X_Train)
print(X_Test)
print(X_Dev)

print(Y_Train)
print(Y_Test)
print(Y_Dev)

# Put all the above in function 

[[50 13  0 ...  1  0  0]
 [38 40  0 ...  1  0  0]
 [53 40  0 ...  1  0  0]
 ...
 [48 32  0 ...  0  0  0]
 [20 30  0 ...  1  0  0]
 [64 50  0 ...  1  0  0]]
[[47 40  0 ...  0  0  0]
 [20 25  0 ...  1  0  0]
 [45 40  0 ...  1  0  0]
 ...
 [27 38  0 ...  1  0  0]
 [40 40  0 ...  1  0  0]
 [52 40  0 ...  1  0  0]]
[[40 60  0 ...  1  0  0]
 [24 50  0 ...  1  0  0]
 [29 58  0 ...  1  0  0]
 ...
 [20 40  0 ...  1  0  0]
 [22 35  0 ...  1  0  0]
 [58 40  0 ...  1  0  0]]
[-1 -1 -1 ... -1 -1  1]
[-1 -1  1 ... -1  1  1]
[ 1 -1 -1 ... -1 -1 -1]


In [20]:
def perceptron(X, Y, maxIteration):
    print(X.shape[1])
    Weight = np.zeros((X.shape[1]))
#     print(Weight.shape)
    for i in range(maxIteration):
        for j in range(X.shape[0]):
#             print(X[j].shape)
            y_hat = np.sign(np.dot(Weight, X[j]))
            if y_hat == 0:
                yhat = -1
            if y_hat != Y[j]:
                Weight = Weight + Y[j]*X[j]
        
    return Weight

In [21]:
def passiveAggressive(X, Y, maxIteration):
    
    Weight = np.zeros(X.shape[1])
    tau = 0 
    for i in range(maxIteration):
        
        for j in range(X.shape[0]):
            tau = ( 1- np.dot(Y[j],(np.dot(Weight,X[j])))) / (np.dot(X[j],X[j]))
            y_hat = np.sign(np.dot(Weight,X[j]))
            if y_hat == 0:
                yhat = -1
            if y_hat != Y[i]:
               Weight = Weight + tau*Y[j]*X[j]
        
    return Weight

In [22]:
def testPerceptron(X, Y, weight):
    count = 0
    for i in range(X.shape[0]):
#   print(list(X.iloc[i,:]))
        y_hat = np.sign(np.dot(weight,X[i]))
        if y_hat == 0:
            yhat = -1
        if y_hat != Y[i]:
            count = count + 1
        
    accuracy = 1- (count/X.shape[0])
    return accuracy

In [23]:
def svm():
    c = [10**-4,10**-3,10**-2,10**-1,1,10]
    for i in c:
        print("*********",i)
        learning_model = SVC(C =i, kernel ='linear')
        learning_model.fit(X_Train,Y_Train)
        print(learning_model.score(X_Test,Y_Test))

In [24]:
def main():
    accuracy = []
#     Insert encoding function call here
    weight_perceptron = perceptron(X_Train,Y_Train,5)
#     weight_passiveAggressive = passiveAggressive(X_Train,Y_Train,5)
    accuracy.append(testPerceptron(X_Train,Y_Train,weight_perceptron))
    print(accuracy)
    print(weight_perceptron)
#     print(weight_passiveAggressive)
#     svm()
# Basic plt
#     plt.plot(weight_perceptron)

In [25]:
main()

94
[0.45992]
[ 3.700e+01  5.500e+01  1.490e+02 -3.520e+02 -5.230e+02  2.910e+02
 -4.920e+02 -2.560e+02 -1.300e+01 -4.020e+02 -4.600e+02 -1.500e+02
 -1.210e+02 -2.530e+02 -4.940e+02 -2.970e+02 -1.040e+02  1.400e+01
  5.590e+02  4.300e+02 -8.140e+02  6.770e+02 -3.000e+01  5.590e+02
 -3.100e+02 -6.880e+02  2.700e+01  1.483e+03 -1.550e+02 -1.280e+03
 -2.940e+02 -2.890e+02 -1.400e+02  0.000e+00 -4.060e+02  9.120e+02
 -5.990e+02 -4.870e+02 -5.220e+02 -8.340e+02 -5.700e+01  7.650e+02
  1.300e+01  2.090e+02  1.860e+02 -2.360e+02 -7.500e+01 -2.540e+02
 -4.130e+02 -1.490e+02 -3.050e+02 -8.730e+02 -3.230e+02 -1.000e+00
 -9.000e+00 -4.700e+01 -3.200e+01 -1.200e+01 -4.700e+01 -3.000e+00
 -4.500e+01  2.500e+01  1.400e+01  2.500e+01 -6.000e+00 -4.500e+01
 -8.000e+00  0.000e+00  1.000e+00 -1.000e+00  4.000e+00 -2.400e+01
 -6.000e+00  1.000e+00  1.000e+00 -8.000e+00 -2.500e+01 -1.100e+01
 -4.090e+02 -1.400e+01 -7.000e+00 -2.800e+01 -3.900e+01 -2.200e+01
 -1.700e+01 -6.100e+01 -1.000e+01 -5.300e+01 -4.0